In [1]:
import numpy as np
import xarray as xr
import pandas as pd

import h5py
import os,json

In [2]:
%load_ext autoreload
%autoreload 2

from icepyx import is2class as ipd

### Choose a region for subsetting as well. Use the same region as in the core demo.

In [3]:
region_a = ipd.Icesat2Data('ATL09',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59', version='2')

In [4]:
session=region_a.earthdata_login('liuzheng','liuzheng@apl.uw.edu')

Earthdata Login password: ········


### Now, generate variable dictionary. 
Get the variable dictionary by parsing the dataset xml information from NSIDC, by calling ```_get_custom_options(session)```. 

The data variables are stored in ```region_a.variables```. 

In [5]:
opts = region_a._get_custom_options(session)
print(region_a.variables)

{'ds_surf_type': ['/ds_surf_type'], 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'], 'control': ['ancillary_data/control'], 'data_end_utc': ['ancillary_data/data_end_utc'], 'data_start_utc': ['ancillary_data/data_start_utc'], 'end_cycle': ['ancillary_data/end_cycle'], 'end_delta_time': ['ancillary_data/end_delta_time'], 'end_geoseg': ['ancillary_data/end_geoseg'], 'end_gpssow': ['ancillary_data/end_gpssow'], 'end_gpsweek': ['ancillary_data/end_gpsweek'], 'end_orbit': ['ancillary_data/end_orbit'], 'end_region': ['ancillary_data/end_region'], 'end_rgt': ['ancillary_data/end_rgt'], 'granule_end_utc': ['ancillary_data/granule_end_utc'], 'granule_start_utc': ['ancillary_data/granule_start_utc'], 'qa_at_interval': ['ancillary_data/qa_at_interval'], 'release': ['ancillary_data/release'], 'start_cycle': ['ancillary_data/start_cycle'], 'start_delta_time': ['ancillary_data/start_delta_time'], 'start_geoseg': ['ancillary_data/start_geoseg'], 'start_gpssow': ['ancillary_data/start_gp

#### Setup the user provided variable list to subset variables
Choose ```latitude``` for ```profile_1``` only for demo purpose. 

```sc_orient``` provide info on beam strength and is stored under ```orbit_info```

In [6]:
subset_kws = {'kw1_list':['profile_1','orbit_info'],'kw2_list':['high_rate'],'var_list':['latitude','sc_orient'],
                                             'add_default_vars':True}

In [7]:
cv09 = region_a.build_subset_coverage(**subset_kws)
print(cv09)

/ancillary_data/atlas_sdp_gps_epoch,/ancillary_data/control,/ancillary_data/data_end_utc,/ancillary_data/data_start_utc,/ancillary_data/end_cycle,/ancillary_data/end_delta_time,/ancillary_data/end_geoseg,/ancillary_data/end_gpssow,/ancillary_data/end_gpsweek,/ancillary_data/end_orbit,/ancillary_data/end_region,/ancillary_data/end_rgt,/ancillary_data/granule_end_utc,/ancillary_data/granule_start_utc,/ancillary_data/qa_at_interval,/ancillary_data/release,/ancillary_data/start_cycle,/ancillary_data/start_delta_time,/ancillary_data/start_geoseg,/ancillary_data/start_gpssow,/ancillary_data/start_gpsweek,/ancillary_data/start_orbit,/ancillary_data/start_region,/ancillary_data/start_rgt,/ancillary_data/version,/ancillary_data/atmosphere/aclr_use_atlas,/ancillary_data/atmosphere/alpha,/ancillary_data/atmosphere/a_m1,/ancillary_data/atmosphere/a_m2,/ancillary_data/atmosphere/asr_cal_factor,/ancillary_data/atmosphere/atlas_bandpass_fw,/ancillary_data/atmosphere/atlas_tele_fov,/ancillary_data/atm

### Setting params and download

In [8]:
region_a.build_CMR_params()
region_a.build_reqconfig_params('download')

In [9]:
region_a.build_subset_params(**{'Coverage':cv09})
region_a.subsetparams


{'time': '2019-02-22T00:00:00,2019-02-28T23:59:59',
 'bbox': '-55,68,-48,71',
 'Coverage': '/ancillary_data/atlas_sdp_gps_epoch,/ancillary_data/control,/ancillary_data/data_end_utc,/ancillary_data/data_start_utc,/ancillary_data/end_cycle,/ancillary_data/end_delta_time,/ancillary_data/end_geoseg,/ancillary_data/end_gpssow,/ancillary_data/end_gpsweek,/ancillary_data/end_orbit,/ancillary_data/end_region,/ancillary_data/end_rgt,/ancillary_data/granule_end_utc,/ancillary_data/granule_start_utc,/ancillary_data/qa_at_interval,/ancillary_data/release,/ancillary_data/start_cycle,/ancillary_data/start_delta_time,/ancillary_data/start_geoseg,/ancillary_data/start_gpssow,/ancillary_data/start_gpsweek,/ancillary_data/start_orbit,/ancillary_data/start_region,/ancillary_data/start_rgt,/ancillary_data/version,/ancillary_data/atmosphere/aclr_use_atlas,/ancillary_data/atmosphere/alpha,/ancillary_data/atmosphere/a_m1,/ancillary_data/atmosphere/a_m2,/ancillary_data/atmosphere/asr_cal_factor,/ancillary_dat

In [10]:
region_a.order_granules(session, verbose=True)

{'feed': {'updated': '2020-03-14T22:11:09.537Z', 'id': 'https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=ATL09&version=002&temporal=2019-02-22T00%3A00%3A00Z%2C2019-02-28T23%3A59%3A59Z&bounding_box=-55%2C68%2C-48%2C71&page_size=10&page_num=1', 'title': 'ECHO granule metadata', 'entry': [{'producer_granule_id': 'ATL09_20190222003738_08490201_002_01.h5', 'time_start': '2019-02-22T00:37:37.000Z', 'orbit': {'ascending_crossing': '130.68730694092687', 'start_lat': '0', 'start_direction': 'A', 'end_lat': '0', 'end_direction': 'A'}, 'updated': '2019-10-31T08:27:54.161Z', 'orbit_calculated_spatial_domains': [{'equator_crossing_date_time': '2019-02-22T00:37:38.252Z', 'equator_crossing_longitude': '130.68730694092687', 'orbit_number': '2437'}], 'dataset_id': 'ATLAS/ICESat-2 L3A Calibrated Backscatter Profiles and Atmospheric Layer Characteristics V002', 'data_center': 'NSIDC_ECS', 'title': 'SC:ATL09.002:166458094', 'coordinate_system': 'ORBIT', 'time_end': '2019-02-22T02:11:55.0

{'feed': {'updated': '2020-03-14T22:11:10.451Z', 'id': 'https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=ATL09&version=002&temporal=2019-02-22T00%3A00%3A00Z%2C2019-02-28T23%3A59%3A59Z&bounding_box=-55%2C68%2C-48%2C71&page_size=10&page_num=2', 'title': 'ECHO granule metadata', 'entry': []}}
Order:  1
[]
Request HTTP response:  201
Order request URL:  https://n5eil02u.ecs.nsidc.org/egi/request?short_name=ATL09&version=002&temporal=2019-02-22T00%3A00%3A00Z%2C2019-02-28T23%3A59%3A59Z&bounding_box=-55%2C68%2C-48%2C71&email=liuzheng%40apl.uw.edu&token=F9AEB039-8155-01A8-B8C7-24190C468993&page_size=10&page_num=1&request_mode=async&include_meta=Y&time=2019-02-22T00%3A00%3A00%2C2019-02-28T23%3A59%3A59&bbox=-55%2C68%2C-48%2C71&Coverage=%2Fancillary_data%2Fatlas_sdp_gps_epoch%2C%2Fancillary_data%2Fcontrol%2C%2Fancillary_data%2Fdata_end_utc%2C%2Fancillary_data%2Fdata_start_utc%2C%2Fancillary_data%2Fend_cycle%2C%2Fancillary_data%2Fend_delta_time%2C%2Fancillary_data%2Fend_geoseg%2C

Retry request status is:  complete


In [11]:
region_a.download_granules(session,'.')

Beginning download of zipped output...
Data request 5000000499635 of  1  order(s) is complete.


### Examine downloaded subset data file 


In [12]:
fn = '166458094/processed_ATL09_20190222003738_08490201_002_01.h5'

#### Check the downloaded dataset
Take ```latitude``` for example,

In [13]:
varname = 'latitude'
#varname = 'sc_orient'

varlist = []
def IS2h5walk(vname, h5node):
    if isinstance(h5node, h5py.Dataset):
        varlist.append(vname)
    return 

with h5py.File(fn,'r') as h5pt:
    h5pt.visititems(IS2h5walk)
    
for tvar in varlist:
    vpath,vn = os.path.split(tvar)
    if vn==varname: print(tvar) 

profile_1/high_rate/latitude


#### Compare the varaible ```latitude``` in the original data and the subsetted dat

In [14]:
region_a.variables['latitude']

['profile_1/high_rate/latitude',
 'profile_1/low_rate/latitude',
 'profile_2/high_rate/latitude',
 'profile_2/low_rate/latitude',
 'profile_3/high_rate/latitude',
 'profile_3/low_rate/latitude']